# PySpark-Based Movie Dataset Exploration

This notebook provides an in-depth exploration of movie information retrieved from the TMDB API using PySpark.

Core operations are handled via helper functions stored in `utility_functions.py`. These cover everything from retrieving and cleaning data to computing KPIs, filtering with custom conditions, examining franchises versus standalone releases, reviewing directors and franchises, and visualizing trends.

In [ ]:
# Load essential packages
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("MovieDataInsight") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [ ]:
from data_extraction import fetch_apis
from utility_functions import *

## Collecting Raw Movie Records

- **Movie ID setup:** List of TMDB movie identifiers prepared for data retrieval.

- **Apply schema:** `SCHEMA()` outlines the structure of the data table.

- **Pull metadata:** Data for each movie is collected using the TMDB API via `fetch_apis()`.

- **Initial overview:** Output includes schema inspection and a sample preview.

In [ ]:
movie_ids = [0, 299534, 19995, 140607, 299536, 597, 135397,
             420818, 24428, 168259, 99861, 284054, 12445,
             181808, 330457, 351286, 109445, 321612, 260513]

schema = SCHEMA()

raw_data = fetch_apis(movie_ids, schema)

print("Raw Dataset Structure:")
raw_data.printSchema()

print("Initial Sample:")
raw_data.show(5, truncate=False)

## Refining Movie Dataset

- **Data transformation:** `clean_data()` reformats columns, handles JSON fields, and casts types.

- **Data sanitization:** Zeros in key metrics like budget or revenue are replaced with nulls.

- **Derived fields:** Profit and ROI columns are computed.

- **Field consistency:** Formats such as genres and countries are standardized.

- **View cleaned records:** Review structure and sample entries.

In [ ]:
cleaned_data = clean_data(raw_data)

cleaned_data.cache()

print("Cleaned Data Schema:")
cleaned_data.printSchema()

print("Preview of Cleaned Dataset:")
cleaned_data.show(5, truncate=False)

## Calculate Financial Metrics
This function calculates financial metrics for each movie:

- Converts budget and revenue to million USD.

- Computes profit_musd as revenue minus budget.

- Computes ROI (Return on Investment) as revenue divided by budget, handling division-by-zero safely.

In [ ]:
print("Financial Metrics: ")
financial_metrics = calculate_financial_metrics(clean_data).select()
financial_metrics.show(truncate=False)

## Extract Director
This function extracts the director's name from the nested credits.crew array for each movie and drops the full credits column to avoid unnecessary data retention.

In [ ]:
print("Extract Directors:")
directors = extract_director(cleaned_data)

## Rank Movies
Ranks movies based on a specified column (e.g., revenue_musd, vote_average) while optionally applying a minimum vote count filter and limiting the result to the top N movies.

In [ ]:
print("Rank movies based on a column: ")
ranked_movies = rank_movies(cleaned_data, "revenue")
ranked_movies.show(5, truncate=False)

## Get Top Directors
**Aggregates key performance indicators for directors:**

- Total number of movies directed.

- Total revenue generated.

- Average rating across their movies.
- Results are sorted by total revenue.

In [ ]:
print("Top Directors:")
top_directors = get_top_directors(cleaned_data)

## Get Top Franchises
**Groups movies by franchise (belongs_to_collection) and calculates:**

- Total movie count.

- Total and average budget/revenue.

- Mean rating.
- Returns the top N highest grossing franchises.

In [ ]:
print("Get top franchises:")
top_franchises = get_top_franchises(cleaned_data)

## Compare Franchise vs Standalone
**Compares performance between franchise movies and standalone titles using key metrics like:**

- Mean revenue, ROI, budget, popularity, and rating.
- A new boolean column is_franchise is added for easy grouping.

In [ ]:
print("Comparing Franchises with Standalone:")
franchise_comparison = compare_franchise_vs_standalone(cleaned_data)
franchise_comparison.show(5, truncate=False)

## Filter Released Movies
Keeps only movies with a status of Released, then drops the status column as it's no longer needed.

In [ ]:
print("Movies Released: ")
movies_released = filter_released_movies(cleaned_data)
movies_released.show(5, truncate=False)

## Filter Valid Movies
**Cleans the dataset by:**

- Removing duplicates and null id/title entries.

- Ensuring each movie has at least 10 non-null attributes, helping maintain high-quality records.

In [ ]:
print("Valid Movies: ")
valid_movies = filter_valid_movies(cleaned_data)
valid_movies.show(5, truncate=False)

## Plot Movie Analysis
**Generates multiple insightful visualizations:**

- Revenue vs Budget: Shows correlation between investment and earnings.

- ROI Distribution by Genre: Box plot to analyze profitability by genre.

- Popularity vs Rating: Correlation between user ratings and social popularity.

- Yearly Trends: Tracks average budget and revenue trends over the years.

- Franchise vs Standalone: Bar chart comparing financial and audience metrics between franchises and standalone movies.

In [ ]:
print("Movie Analysis Graphs:")
graphs = plot_movie_analysis(cleaned_data)
graphs.show(5, truncate=False)